In [4]:
In [1]: %load_ext autoreload

In [2]: %autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Cars Demo

In [7]:
import os
import sqlite3 as lite

filename = "cars.db"

con = lite.connect(filename)

with con:
    cur = con.cursor()

    # Create table
    cur.execute("CREATE TABLE cars(id INT, name TEXT, price INT)")
    
    # Insert data
    cur.execute("INSERT INTO cars VALUES(1,'Audi',52642)")
    cur.execute("INSERT INTO cars VALUES(2,'Mercedes',57127)")
    cur.execute("INSERT INTO cars VALUES(3,'Skoda',9000)")
    cur.execute("INSERT INTO cars VALUES(4,'Volvo',29000)")
    cur.execute("INSERT INTO cars VALUES(5,'Bentley',350000)")
    cur.execute("INSERT INTO cars VALUES(6,'Citroen',21000)")
    cur.execute("INSERT INTO cars VALUES(7,'Hummer',41400)")
    cur.execute("INSERT INTO cars VALUES(8,'Volkswagen',21600)")
    
    # Fetch data
    print("Data:")
    print(cur.execute("SELECT * FROM CARS").fetchall())
    
    # Assert that a file was made
    assert os.path.exists('cars.db')
    os.remove(filename)

Data:
[(1, 'Audi', 52642), (2, 'Mercedes', 57127), (3, 'Skoda', 9000), (4, 'Volvo', 29000), (5, 'Bentley', 350000), (6, 'Citroen', 21000), (7, 'Hummer', 41400), (8, 'Volkswagen', 21600)]


# Crawler Database Demo

In [73]:
import contextlib
import os
import random
import sqlite3
import threading
import time

def execute_statement(statement, args={}):
    with contextlib.closing(sqlite3.connect("crawler.db")) as conn: # auto-closes
        with conn: # auto-commits
            with contextlib.closing(conn.cursor()) as cursor: # auto-closes
                return cursor.execute(statement, args).fetchall()

In [74]:
import sqlite3

def create_table(remove=False):
    filename = "crawler.db"
    
    try:
        os.remove(filename)
    except Exception as e:
        print(e)
        pass

    q = """
    CREATE TABLE observations (
        services INT,
        timestamp INT,
        receiver_services INT,
        receiver_ip TEXT,
        receiver_port INT,
        sender_services INT,
        sender_ip TEXT,
        sender_port INT,
        nonce INT,
        user_agent TEXT,
        latest_block INT,
        relay INT
    )
    """
    execute_statement(q)

In [75]:
def observe_node(args_dict):
    q = """
    INSERT INTO observations (
        services,
        timestamp,
        receiver_services,
        receiver_ip,
        receiver_port,
        sender_services,
        sender_ip,
        sender_port,
        nonce,
        user_agent,
        latest_block,
        relay
    ) VALUES (
        :services,
        :timestamp,
        :receiver_services,
        :receiver_ip,
        :receiver_port,
        :sender_services,
        :sender_ip,
        :sender_port,
        :nonce,
        :user_agent,
        :latest_block,
        :relay
    )
    """
    execute_statement(q, args_dict)

In [77]:
from io import BytesIO

from crawler1 import NetworkEnvelope, VersionMessage

ver = b'\xf9\xbe\xb4\xd9version\x00\x00\x00\x00\x00z\x00\x00\x00\x83\xa5\x04\xc2\x7f\x11\x01\x00\r\x04\x00\x00\x00\x00\x00\x00wuZ\\\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\x8aD\xabn\xa8 \r\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x008;l\xe5\xe7\xea\xa5a$/Satoshi:0.17.0(UASF-SegWit-BIP148)/\\\x92\x08\x00\x01'

# bytes -> python -> sql
ne = NetworkEnvelope.parse(BytesIO(ver))
vm = VersionMessage.parse(BytesIO(ne.payload))
args_dict = vm.__dict__
print(args_dict)
print()

create_table()
observe_node(args_dict)

print(execute_statement("select * from observations"))

[(1037, 1549432183, 0, '::ffff:138.68.171.110', 43040, 1037, '0.0.0.0', 0, 7036288274543688504, b'/Satoshi:0.17.0(UASF-SegWit-BIP148)/', 561756, 1)]
